# Crime_Project
### GeoMapping
* Dependencies and Setup

In [9]:
import gmaps
import pandas as pd
import gmaps.datasets
import gmaps.geojson_geometries
import matplotlib.pyplot as plt
from matplotlib.cm import viridis, plasma
from matplotlib.colors import to_hex

import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

# Census & gmaps API Keys
from config import census_key, g_key

# Configure gmaps
gmaps.configure(api_key=g_key)
states_geojson = gmaps.geojson_geometries.load_geometry('us-states')
# print(len(states_geojson['features']))
# print(states_geojson['features'][0])

In [15]:
# Create GmapsAnalysis Object
class GmapsAnalysis():
  
    # Define initialization method
    def __init__(self, data_dict, geometry='us-states'):
        # We will need to scale the states values to lie between 0 and 1
        self.data = data_dict
        self.min = min(self.data.values())
        self.max = max(self.data.values())
        self.range = self.max - self.min
        self.geojson = gmaps.geojson_geometries.load_geometry(geometry)
        self.fig = gmaps.figure()
        
    def calculate_color(self, state, cmap_color = plasma):
        # make state a number between 0 and 1
        normalized_state = (state - self.min) / self.range
        # invert state so that high value gives dark color
        inverse_state = 1.0 - normalized_state
        # transform the state value to a matplotlib color
        mpl_color = cmap_color(inverse_state)
        # transform from a matplotlib color to a valid CSS color
        gmaps_color = to_hex(mpl_color, keep_alpha=False)

        return gmaps_color

    def colors_list(self, cmap_color = plasma):
        colors = []
        for feature in self.geojson['features']:
            geo_state_name = feature['properties']['NAME']
            try:
                state = self.data[geo_state_name]
                color = self.calculate_color(state, cmap_color)
            except KeyError:
            # no value for that state: return default color
                color = (0, 0, 0, 0.3)
            colors.append(color)
        return colors

    def color_map(self, cmap_color = plasma, opacity = 0.8):
        states_layer = gmaps.geojson_layer(self.geojson,
                                           fill_color = self.colors_list(cmap_color), stroke_color = self.colors_list(),
                                            fill_opacity = opacity)
        self.fig.add_layer(states_layer)
        
    def symbols_map(self, symbols_data, color = 'blue', scale = 1):
        symbols_layer = gmaps.symbol_layer(symbols_data, fill_color = color, stroke_color = color, scale = scale)
        self.fig.add_layer(symbols_layer)
        
    def marker_map(self, locations = [(41.8781136, -87.6297982)], cont = 'Chicago, IL'):
        marker = gmaps.marker_layer(locations, info_box_content= cont)
        self.fig.add_layer(marker)       

### ColorMap for 51 US states based on adjusted rate for Crime and Deaths (location of 788 US cities added on map)

In [23]:
# Creating US states Crime Map
data = pd.read_csv('resources/states_2011.csv')
data_dict = data.filter(['State_Name', 'Crime_Adj'])
data_dict = data_dict.set_index('State_Name').to_dict()["Crime_Adj"]
crime = GmapsAnalysis(data_dict)
crime.color_map(cmap_color = plasma)
# Adding US cities symbols and Chicago infobox
cities_data = pd.read_csv('resources/uscities_data.csv')
cities_geo = cities_data[['Lat', 'Lng']]
crime.symbols_map(cities_geo)
crime.marker_map()
crime.fig

<bound method GmapsAnalysis.color_map of <__main__.GmapsAnalysis object at 0x7fc04f8e4be0>>

In [ ]:
# Creating US states Deaths Map

### HeatingMap for 79 US cities

In [19]:
# import Murders stat Dataset for 79 US cities
murders_data = pd.read_csv('resources/cities_murders.csv')
chicago = murders_data[murders_data['City']=='Chicago']
chicago_locations = chicago[['Lat', 'Lng']]
murders_data.head()

,City,State,2014,2015,% Change,Lat,Lng
0,Chicago,Illinois,411,478,67,41.878114,-87.629798
1,New York,New York,333,352,19,40.712775,-74.005973
2,Baltimore,Maryland,211,344,133,39.290385,-76.612189
3,Houston,Texas,242,303,61,29.760427,-95.369803
4,Detroit,Michigan,298,295,-3,42.331427,-83.045754


In [3]:
# Creating DataSet for HeatMap Layers
fig = gmaps.figure()
locations = murders_data[['Lat', 'Lng']]
weight = murders_data['2015']
fig.add_layer(gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=30, point_radius=1))
marker_chi = gmaps.marker_layer(chicago_locations,
    info_box_content= 'Chicago,IL 414 murders in 2020')
fig.add_layer(marker_chi)
fig

Figure(layout=FigureLayout(height='420px'))